### swarm-master ###

The script similates work of large number of torrent clients (swarm)
It prepares the set of torrent files and launches lt-client

Please edit ```input_file``` and ```workingDir``` variables to make script work properly.

To install dependices use commands: 
```
sudo apt-get install ctorrent
pip install numpy 
pip install tqdm
```

In [ ]:
import random
import math
import numpy as np
from tqdm import tqdm
from time import time
import time

In [ ]:
!pwd #what is our working directorty?

In [ ]:
n = 100
base_port = 6000
input_file = '/home/latna/aponom/swarm-master/data/stackoverflow.10kk_short.tsv'
workingDir = '/home/latna/artsokol/libtorrent/examples/.libs/'
dataDir = '../../data/'
torrentsDir = '../../data/torrents/'
nDigits = int(math.log10(n) + 1); #needs for format purpose
%cd $workingDir

In [ ]:
#nw = [ int(f) for f in  np.random.normal(size=n, scale=50, loc = median)] 

In [ ]:
texts = []
with open(input_file) as infile:
    for line in tqdm(infile):
        texts.append(line)

In [ ]:
torrentsDir

In [ ]:
#create torrentsDir if it doesn't exist, clean it, and generate n torrents!
!rm -rf $torrentsDir
!mkdir -p $torrentsDir
#generate torrents
for i in tqdm(range(n)):
    zi = str(i).zfill(nDigits)
    !echo $zi > {dataDir}stubFile.txt
    command = 'ctorrent -t -u http://tracker.example.com:6969/announce -s ' + torrentsDir + 'swarm_'+ zi + '.torrent '+  dataDir + 'stubFile.txt -c'\
    + "'"  + texts[i] + "'"
    out = !$command

In [ ]:
torrents = !ls {torrentsDir + '*.torrent'}
torrents[:5]

In [ ]:
#clean all tmux sessions
!tmux kill-server
#clean .resume dir
!rm -f ./.resume/*

In [ ]:
#start gate node
!tmux new -d -s gate-node './lt-client_test ../../data/gate1.torrent  -p 26401 -g'
time.sleep(5)

In [ ]:
session_names = []
lt_client_start_commands = [] 
port = base_port
for torrent in tqdm(torrents):
    session_name = torrent.split('/')[-1].split('.')[0]
    session_names.append(session_name)
    command = "'./lt-client_test " + torrent + " -p " + str(port) + "'"
    lt_client_start_command = 'tmux new -d -s ' + session_name + ' ' + command;
    lt_client_start_commands.append(lt_client_start_command)
    port = port + 1

In [ ]:
#start all lt-clients
%cd $workingDir
for lt_client_start_command in tqdm(lt_client_start_commands):
    !$lt_client_start_command
    time.sleep(10)

In [ ]:
#stop all sessions
for session in tqdm(session_names):
    stop_session(session)

In [ ]:
def stop_session(session_name):
    !tmux kill-session -t $session_name
    

In [ ]:
session_names = []
for torrent in tqdm(torrents):
    session_name = torrent.split('/')[-1].split('.')[0]
    session_names.append(session_name)

In [ ]:
lt_client_start_commands[:5]